## Hamiltonian CNN?

In [1]:
import os as os
import numpy as np
import sklearn.datasets as datasets
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
import torch.utils.data as data_utils
from torchdiffeq import odeint
from torchsummary import summary
import torchvision as torchvision

from torchcontrol.arch_cpugpu import HDNN
from torchcontrol.predictors import MLP, CNN
from torchcontrol.utils import genpoints, dump_tensors

#MNIST imports
import torchvision.transforms as transforms
import torchvision.datasets as dset

In [2]:
dump_tensors()

Total size: 0


In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
root = './data'
if not os.path.exists(root):
    os.mkdir(root)
    
trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (1.0,))])
# if not exist, download mnist dataset
train_set = dset.MNIST(root=root, train=True, transform=trans, download=True)
test_set = dset.MNIST(root=root, train=False, transform=trans, download=True)

batch_size = len(train_set)

trainloader = torch.utils.data.DataLoader(
                 dataset=train_set,
                 batch_size=10000,
                 shuffle=True)
testloader = torch.utils.data.DataLoader(
                dataset=test_set,
                batch_size=10000,
                shuffle=False)

In [5]:
d = iter(testloader)

In [6]:
x = next(d)

In [7]:
x[0].shape

torch.Size([10000, 1, 28, 28])

## CNN model

In [8]:
class CNN(nn.Module):
    def __init__(self, conv_layers, dense_layers, smax_l = True):
        '''
        smax_l: leave True for softmax applied to ouput
        '''
        super().__init__()
        self.conv_layers = nn.ModuleList([nn.Conv1d(conv_layers[i], conv_layers[i + 1], kernel_size = 3) 
                                     for i in range(len(conv_layers) - 1)])
        self.dense_layers = nn.ModuleList([nn.Linear(dense_layers[i], dense_layers[i + 1]) 
                                     for i in range(len(dense_layers) - 1)])

        self.max = nn.MaxPool1d(2)
        #self.smax = smax_l
        #self.bn = nn.ModuleList([nn.BatchNorm1d(conv_layers[i]) for i in range(len(conv_layers)-1)])
        
    def forward(self, x):
        x = x[0].view(x.size(0), 1, -1)
        for i,l in enumerate(self.conv_layers):
            x = l(x)
            #x = self.bn[i](x)
            x = F.relu(x)
            x = self.max(x)
        x = x.view(x.size(0), -1)
        for l in self.dense_layers:
            l_x = l(x)
            x = F.relu(l_x)
        if self.smax: return F.log_softmax(l_x, dim=-1)
        else: return torch.sigmoid(l_x)

## HCNN

In [22]:
m = HDNN('CNN',[[1,3,5],[3900,10]],[1,2,0],1,'cpu').to(device)

In [23]:
m.predictor

CNN(
  (conv_layers): ModuleList(
    (0): Conv1d(1, 3, kernel_size=(3,), stride=(1,))
    (1): Conv1d(3, 5, kernel_size=(3,), stride=(1,))
  )
  (dense_layers): ModuleList(
    (0): Linear(in_features=3900, out_features=10, bias=True)
  )
  (max): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)

In [24]:
summary(m.predictor,(1,1,28*28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1               [-1, 3, 782]              12
            Conv1d-2               [-1, 5, 780]              50
            Linear-3                   [-1, 10]          39,010
Total params: 39,072
Trainable params: 39,072
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.05
Params size (MB): 0.15
Estimated Total Size (MB): 0.20
----------------------------------------------------------------


In [15]:
m.predictor.state_dict()

OrderedDict([('conv_layers.0.weight', tensor([[[ 0.1813, -0.4917,  0.2969]],
              
                      [[ 0.2819,  0.4856, -0.4259]],
              
                      [[ 0.0112, -0.2719,  0.0653]]], device='cuda:0')),
             ('conv_layers.0.bias',
              tensor([-0.5022, -0.2418, -0.2881], device='cuda:0')),
             ('conv_layers.1.weight', tensor([[[-0.2179, -0.0615,  0.1609],
                       [-0.2973,  0.2313,  0.1312],
                       [ 0.1769,  0.0275,  0.0111]],
              
                      [[-0.1518,  0.2178, -0.1435],
                       [ 0.0569, -0.1953,  0.2118],
                       [-0.1228, -0.2317,  0.1804]],
              
                      [[ 0.0130, -0.1297, -0.0028],
                       [-0.2641, -0.1492,  0.1353],
                       [-0.3276,  0.0714,  0.2682]],
              
                      [[ 0.1062,  0.1043, -0.1449],
                       [ 0.2639, -0.3310, -0.0120],
                  

## Hope and dreams

In [16]:
print('Initial accuracy on training dataset: {} %'.format(m.pred_accuracy(trainloader)))

Initial accuracy on training dataset: 0.0987 %


In [25]:
m.fit(trainloader,3,time_delta=None,iter_accuracy=20,ode_t=10,ode_step=2)

odeint iter: 10 
odeint iter: 20 
odeint iter: 30 
odeint iter: 40 
odeint iter: 50 
odeint iter: 60 
odeint iter: 70 
odeint iter: 80 
odeint iter: 90 
odeint iter: 100 
odeint iter: 110 
odeint iter: 120 
odeint iter: 130 
odeint iter: 140 
odeint iter: 150 
odeint iter: 160 
odeint iter: 170 
odeint iter: 180 
odeint iter: 190 
odeint iter: 200 
odeint iter: 210 
odeint iter: 220 
odeint iter: 230 
odeint iter: 240 
odeint iter: 250 
odeint iter: 260 
odeint iter: 270 
odeint iter: 280 
odeint iter: 290 
odeint iter: 300 
odeint iter: 310 
odeint iter: 320 
odeint iter: 330 
odeint iter: 340 
odeint iter: 350 
odeint iter: 360 
odeint iter: 370 
odeint iter: 380 
odeint iter: 390 
odeint iter: 400 
odeint iter: 410 
odeint iter: 420 
odeint iter: 430 
odeint iter: 440 
odeint iter: 450 
odeint iter: 460 
odeint iter: 470 
odeint iter: 480 
odeint iter: 490 
odeint iter: 500 
odeint iter: 510 
odeint iter: 520 
odeint iter: 530 
odeint iter: 540 
odeint iter: 550 
odeint iter: 560 
o

odeint iter: 4370 
odeint iter: 4380 
odeint iter: 4390 
odeint iter: 4400 
odeint iter: 4410 
odeint iter: 4420 
odeint iter: 4430 
odeint iter: 4440 
odeint iter: 4450 
odeint iter: 4460 
odeint iter: 4470 
odeint iter: 4480 
odeint iter: 4490 
odeint iter: 4500 
odeint iter: 4510 
odeint iter: 4520 
odeint iter: 4530 
odeint iter: 4540 
odeint iter: 4550 
odeint iter: 4560 
odeint iter: 4570 
odeint iter: 4580 
odeint iter: 4590 
odeint iter: 4600 
odeint iter: 4610 
odeint iter: 4620 
odeint iter: 4630 
odeint iter: 4640 
odeint iter: 4650 
odeint iter: 4660 
odeint iter: 4670 
odeint iter: 4680 
odeint iter: 4690 
odeint iter: 4700 
odeint iter: 4710 
odeint iter: 4720 
odeint iter: 4730 
odeint iter: 4740 
odeint iter: 4750 
odeint iter: 4760 
odeint iter: 4770 
odeint iter: 4780 
odeint iter: 4790 
odeint iter: 4800 
odeint iter: 4810 
odeint iter: 4820 
odeint iter: 4830 
odeint iter: 4840 
odeint iter: 4850 
odeint iter: 4860 
odeint iter: 4870 
odeint iter: 4880 
odeint iter:

odeint iter: 8680 
odeint iter: 8690 
odeint iter: 8700 
odeint iter: 8710 
odeint iter: 8720 
odeint iter: 8730 
odeint iter: 8740 
odeint iter: 8750 
odeint iter: 8760 
odeint iter: 8770 
odeint iter: 8780 
odeint iter: 8790 
odeint iter: 8800 
odeint iter: 8810 
odeint iter: 8820 
odeint iter: 8830 
odeint iter: 8840 
odeint iter: 8850 
odeint iter: 8860 
odeint iter: 8870 
odeint iter: 8880 
odeint iter: 8890 
odeint iter: 8900 
odeint iter: 8910 
odeint iter: 8920 
odeint iter: 8930 
odeint iter: 8940 
odeint iter: 8950 
odeint iter: 8960 
odeint iter: 8970 
odeint iter: 8980 
odeint iter: 8990 
odeint iter: 9000 
odeint iter: 9010 
odeint iter: 9020 
odeint iter: 9030 
odeint iter: 9040 
odeint iter: 9050 
odeint iter: 9060 
odeint iter: 9070 
odeint iter: 9080 
odeint iter: 9090 
odeint iter: 9100 
odeint iter: 9110 
odeint iter: 9120 
odeint iter: 9130 
odeint iter: 9140 
odeint iter: 9150 
odeint iter: 9160 
odeint iter: 9170 
odeint iter: 9180 
odeint iter: 9190 
odeint iter:

In [26]:
print('Final accuracy on test dataset: {} %'.format(m.pred_accuracy(testloader)))

Final accuracy on test dataset: 0.9128 %
